# Preserving Database Load Statistics as part of a migration from OnPrem to Azure

This exercise aims to set up a process to capture key performance and load metrics of your SQL Server databases before and after migrating them from on-premises to Azure. The metrics being captured include read requests, reads, writes, and IO stalls for each database file. This information is essential for understanding how our systems behave under normal conditions on-premises and to establish a performance baseline.

By scheduling a job to run every minute, we will have a comprehensive, high-resolution picture of the performance of each database in our system. This minute-by-minute snapshot will give us a granular look at how your system performs throughout the day, capturing peak loads, idle times, and everything in between.

Once the migration to Azure is complete, we will continue to capture this information from Azure SQL instances and have some baseline to compare to when we have to compare performance between onPrem and Azure Environments. This will help us highlight any performance improvements, regressions, or changes in load patterns. This will not only validate the effectiveness of the migration but also help in tuning and optimizing the databases in the new environment.


# Setup
To set this up we will need following new objects in a DBAUtility Like Database.
* dbo.IOStats
* dbo.CaptureIOStats
* New SQL Task Scheduler Job

## Setup New Table and Stored Procedure
Following query will create the table and stored procedure objects required for this 



In [ ]:
CREATE TABLE dbo.IOStats
(
    RecordID int IDENTITY(1,1) PRIMARY KEY,
    DatabaseName nvarchar(128),
    DatabaseId smallint,
    Filename nvarchar(260),
    TotalReadRequests bigint,
    TotalReads bigint,
    TotalWrites bigint,
    IOStall bigint,
    DateRecorded datetime
)
GO 



CREATE OR ALTER PROCEDURE dbo.CaptureIOStats
AS
BEGIN
    INSERT INTO dbo.IOStats (DatabaseName, DatabaseId, Filename, TotalReadRequests, TotalReads, TotalWrites, IOStall, DateRecorded)
    SELECT 
        DB_NAME(fs.database_id) AS DatabaseName,
        fs.database_id AS DatabaseId,
        mf.name AS Filename,
        fs.num_of_reads AS TotalReadRequests,
        fs.num_of_bytes_read AS TotalReads,
        fs.num_of_bytes_written AS TotalWrites,
        fs.io_stall AS IOStall,
        GETDATE() AS DateRecorded
    FROM sys.dm_io_virtual_file_stats(NULL, NULL) AS fs
    JOIN sys.master_files AS mf
    ON fs.database_id = mf.database_id AND fs.file_id = mf.file_id;
END
GO

## Setup: SQL Agent Job
Job will run the following query on a DBAUtility like database 
`exec dbo.CaptureIOStats`

| Property | Value | 
| --- | --- | 
| Frequency | Every One Minute | 
| Scheduled to/from | 1201 am to 1159 pm daily, 7 days a week | 
| Which Instances to include | All SQL server Instances that are part of the On-Orem to Azure Migration | 
| Alerts | All job failure alerts to be sent to psdata@rsimail.com and corresponding tech team group | 


# Storage needs, Data Cleanup, MISC.

## Size

Rough estimate, for 30 day for a regular project is under 5GB per instance.

| Every Minute | Minutes in and Hour | Hours in a Day | 30 days | 588 Bytes per Row | Total Bytes | Total GB |
| --- | --- | --- | --- | --- | --- | --- |
| 200 | 60 | 24 | 30 | 588 | 4,819,840,000 bytes | 4.48 GB |

```

RecordID (int) - 4 bytes DatabaseName (varchar(128)) - 2 bytes per character (assuming non-Unicode), so at most 256 bytes. 

DatabaseId (smallint) - 2 bytes Filename (varchar(128)) - 256 bytes if you use the maximum length. 

TotalReadRequests (bigint) - 8 bytes 

TotalReads (bigint) - 8 bytes 

TotalWrites (bigint) - 8 bytes 

IOStall (bigint) - 8 bytes 

DateRecorded (datetime) - 8 bytes

So, the total is 4 + 256 + 2 + 256 + 8 + 8 + 8 + 8 + 8 = 558 bytes per row.

30 days for 200 files, the number of rows would be: 60 (minutes/hour) x 24 (hours/day) x 30 (days) x 200 (files) = 8,640,000 rows.

So the total size required for the IOStats table would be approximately 558 (bytes/row) x 8,640,000 (rows) = 4,819,840,000 bytes or approximately 4.48 GB.

```

# How to make sense of this data once it is collected

You can use the following query to view data collected. Goal right now is to preserve the data and have some baseline to compare to once we are in Azure. 


In [ ]:
WITH GroupedStats AS (
    SELECT 
        DatabaseName, 
        DateRecorded,
        SUM(TotalReadRequests) AS TotalReadRequests,
        SUM(TotalReads) AS TotalReads,
        SUM(TotalWrites) AS TotalWrites,
        SUM(IOStall) AS IOStall
    FROM 
        dbo.IOStats
    GROUP BY 
        DatabaseName, 
        DateRecorded
)

SELECT 
    DatabaseName, 
    DateRecorded,
    TotalReadRequests - LAG(TotalReadRequests, 1, 0) OVER (PARTITION BY DatabaseName ORDER BY DateRecorded) AS DeltaReadRequests,
    TotalReads - LAG(TotalReads, 1, 0) OVER (PARTITION BY DatabaseName ORDER BY DateRecorded) AS DeltaReads,
    TotalWrites - LAG(TotalWrites, 1, 0) OVER (PARTITION BY DatabaseName ORDER BY DateRecorded) AS DeltaWrites,
    IOStall - LAG(IOStall, 1, 0) OVER (PARTITION BY DatabaseName ORDER BY DateRecorded) AS DeltaIOStall,
    DATEDIFF(second, LAG(DateRecorded, 1) OVER (PARTITION BY DatabaseName ORDER BY DateRecorded), DateRecorded) AS SecondsDiff
FROM 
    GroupedStats
ORDER BY 
    DatabaseName, 
    DateRecorded

# End Goal - Cutover
It is expected as part of the final cutover from OnPrem to Azure, we will have to backup and migrate the database containing this table or this table on its own to Azure, from all relevant instances.